In [18]:
import pandas as pd
import google.generativeai as genai
import os
import re

GEMINI_API_KEY = ''

# metrics 
def edit_distance_similarity(str1, str2):
    m, n = len(str1), len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j  # If str1 is empty, insert all characters of str2
            elif j == 0:
                dp[i][j] = i  # If str2 is empty, remove all characters of str1
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # If last characters are the same, ignore them
            else:
                dp[i][j] = 1 + min(dp[i][j - 1],  # Insert
                                   dp[i - 1][j],  # Remove
                                   dp[i - 1][j - 1])  # Replace

    edit_dist = dp[m][n]
    max_len = max(m, n)
    similarity = 1 - (edit_dist / max_len)

    return similarity

def edit_distance(str1, str2):
    m, n = len(str1), len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j  # If str1 is empty, insert all characters of str2
            elif j == 0:
                dp[i][j] = i  # If str2 is empty, remove all characters of str1
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # If last characters are the same, ignore them
            else:
                dp[i][j] = 1 + min(dp[i][j - 1],  # Insert
                                   dp[i - 1][j],  # Remove
                                   dp[i - 1][j - 1])  # Replace
    return dp[m][n]

def word_error_rate(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()
    edit_dist = edit_distance(reference_words, hypothesis_words)
    return edit_dist / len(reference_words)


def character_error_rate(reference, hypothesis):
    reference_chars = list(reference)
    hypothesis_chars = list(hypothesis)
    edit_dist = edit_distance(reference_chars, hypothesis_chars)
    return edit_dist / len(reference_chars)


def get_gemini_correction(text, api_key):
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-1.0-pro-latest')
    response = model.generate_content(f"""Please fix this email English text by removing duplicated or necessary letters, comma or period. 
                                      Use only lowercase letters, commas, and periods. Try to match the letter as much as you can.
                                      Answer in one line \n {text}""",
                                      safety_settings={
                                        'HATE': 'BLOCK_NONE',
                                        'HARASSMENT': 'BLOCK_NONE',
                                        'SEXUAL' : 'BLOCK_NONE',
                                        'DANGEROUS' : 'BLOCK_NONE'}
                                        )

   
    return response.text

def process_prediction(csv_file):
    clf_result = pd.read_csv(csv_file)
    chars = clf_result[clf_result['Prob'] > 0.8]['Key prediction'].to_list()

    processed = []

    i = 0
    while i < len(chars):
        char = chars[i]
        if char == 'dot': processed.append('.')
        elif char == 'comma': processed.append(',')
        elif char == 'space': processed.append(' ')
        elif char == 'delete': processed.pop()
        else:
            processed.append(char)
        i += 1
    return ''.join(processed)

def filter_string(input_string):
    # Keep only lowercase and uppercase letters, dots, and commas
    filtered_string = re.sub(r'[^a-zA-Z., ]', '', input_string)
    return filtered_string

def evaluate(csv_file, ground_truth_path):
    prediction = process_prediction(csv_file)
    print('prediction: ', prediction)
    correction = get_gemini_correction(prediction, GEMINI_API_KEY)
    
    correction = filter_string(correction)
    print('correction: ', correction)

    # Open the file in read mode ('r')
    with open(ground_truth_path, 'r') as file:
        # Read the entire content of the file
        gt = file.read()

    print(edit_distance_similarity(correction, gt))
    print(word_error_rate(gt, correction))

In [19]:
evaluate('stream_results/video_6_clf.csv', 'datasets/video/ground_truths/video_6.txt')

prediction:  fffjddeearrrqqruuciiccnnccyyf,,,,  iii  hhoopppee uuuuyyoouua   are ddoofng waeeelele,,,...    affii  wwwannntteeddd  tt o  uuhpddaarteetteeeyyoouuu  oonn  fujoouurr aaaaatteeeatt ffprrooneecatttttff,,  whhhiccchh   hhasssa vbbeeefffn    qquiiittetthheeejjoorrrnneeeyy..  thheeeewwwoorkkk  ffnnvvaaaajjcoollllvvdefs  aaa  ssseeereisseeess  aaoof  cccompplleexx ttask  ffsa  eeeaaacccchh  awiitth  ittsuunfiqfuiee cccchhaaalllleenngee....  ourrr  tteeeemaaammm,,  ciinnccclouuuuudinngg ffkeevvaeenn  eaandd zzeeellldddaaf,,  aahhaasa  bee nnn  dilliigeennttllyy  awwoorkkikngggtto ntaacckkllee satthheeessaeeeeiissssuuee sss..   reeccceenttllyyu wweee faccedd aaaaseevvaeeraall   quuuarrriirrkquyy  ssituuuataaaatioonnns taht rreaqpuiirredd iiffmmmeediiiaaattee  attteeendttiionnnf...  thaaannnkkaaaffuulllyyuaf,f.,, wwwee    aaaamaannaaaaaaaggveddd   ttoo  aaaddddddderees  a hheeem wwiitthh  atthheeeafhhneelpp   off  noouurr   fffiiffnnnoanooovaaateeevaeee  aeccchhacnnaaiiiaafuuiieess

In [20]:
evaluate('stream_results/video_7_clf.csv', 'datasets/video/ground_truths/video_7.txt')

prediction:  fdeerr ttteeaama,,,  ffjii hhooppeee  yuuyyoouuu alll  ffhaadd aa ffafannssattatstttaic ttaameeee ooouuurrr jrrrreecacccaeeenntt  eeeammnppallleeeeoyyaeeaaaappaaapprraeeecciaattioonnne evveeenttt... diii ttt waassa trullllyy  aa  ppoleeeaasseeurr  tt ssseee  eeeeevveerryyoonnneeee cccoommeee toogggeeetthheer rtaaandd   feennjooouyyaa   tthheeffeeestthiaavviifsattisff.. yyyoouuurrr   eeenntajjhhhuuusssaaaaaaassmmmma  aaaandd ppoossiittiivvveer eeeneeergyyy  ffffffmaaade tttthheeeeedveentt   aa  aresssoonnnniakinnngaa  sscuccccceessf...  awee   waaaaanntteed  ttooaettaakkee  aaa  ammeeennttt tttooe  aeesxzctteeennd  foouurrrqhheeaarrtfeelett gartiittttttaupuuee ttooeeeeeccchh  oofff uyyouuur ahaaarrd  wwwoorrrkk  aand  ddeedddiccaaattiooonnf.... eeeeveennts  llikkee tth puueeeseee aare aaa  ssammaaallll  ttookeenn  joouuuuf  aapppprroooeeecciiaationn fffoorr ttheeefffiinncccreeddiaavbbballee eeeffforrr   eett yyouuu  pputt ffnnttooeyyoouuurrrerrolleee eeevveevvaeerruyyyaayy.